In [20]:
import pandas as pd
import numpy as np

# Create the dataset
data = pd.DataFrame({
    'Day': [1, 2, 3, 4, 5, 6, 7],
    'Temperature': [25, 28, 30, 22, 20, 27, 26],
    'Humidity': [70, 80, 90, 60, 50, 75, 75],
    'Pressure': [1010, 1005, 1000, 1015, 1020, 1008, 1012],
    'Wind_Speed': [10, 15, 20, 5, 0, 12, 15],
    'Cloud_Cover': [30, 50, 80, 20, 10, 40, 60],
    'Rainfall': [0, 10, 20, 0, 0, 5, '?']
})

# Convert rainfall to binary (Rain/No Rain)
data['Rain'] = data['Rainfall'].apply(lambda x: 'Yes' if x not in [0, '?'] else 'No')

# Calculate prior probabilities
total_days = len(data[data['Rain'] != '?'])
rain_days = len(data[data['Rain'] == 'Yes'])
no_rain_days = len(data[data['Rain'] == 'No'])

P_rain = rain_days / total_days
P_no_rain = no_rain_days / total_days

# Function to calculate likelihood for a feature
def calculate_likelihood(feature_value, feature_name, rain_condition):
    # Get mean and std for the feature based on rain condition
    subset = data[data['Rain'] == rain_condition][feature_name]
    mean = subset.mean()
    std = subset.std()
    
    # Using Gaussian probability density
    if std == 0:  # Handle zero standard deviation
        return 1 if abs(feature_value - mean) < 0.1 else 0
    exponent = -((feature_value - mean) ** 2) / (2 * (std ** 2))
    return (1 / (std * np.sqrt(2 * np.pi))) * np.exp(exponent)

# Day 7 features
day7_features = {
    'Temperature': 26,
    'Humidity': 75,
    'Pressure': 1012,
    'Wind_Speed': 15,
    'Cloud_Cover': 60
}

# Calculate likelihoods for rain
rain_likelihood = P_rain
no_rain_likelihood = P_no_rain

for feature, value in day7_features.items():
    rain_likelihood *= calculate_likelihood(value, feature, 'Yes')
    no_rain_likelihood *= calculate_likelihood(value, feature, 'No')

# Normalize probabilities
total_likelihood = rain_likelihood + no_rain_likelihood
rain_probability = rain_likelihood / total_likelihood
no_rain_probability = no_rain_likelihood / total_likelihood

prediction = "Rain" if rain_probability > no_rain_probability else "No Rain"
print(f"Prediction for Day 7: {prediction}")
print(f"Probability of Rain: {rain_probability:.2%}")
print(f"Probability of No Rain: {no_rain_probability:.2%}")

Prediction for Day 7: Rain
Probability of Rain: 66.07%
Probability of No Rain: 33.93%
